<a href="https://colab.research.google.com/github/kjedrzejczak/Machine_Learning_Bootcamp_practice_II/blob/main/10_PCA_wine_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

np.set_printoptions(precision=4, suppress=True, edgeitems=5, linewidth=200)

In [2]:

df_raw = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df = df_raw.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [3]:
data = df.iloc[:, 1:]
target = df.iloc[:, 0]
data.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
target.value_counts()

0
2    71
1    59
3    48
Name: count, dtype: int64

Dzielenie danych na zbriót treningowy i testowy

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target)

print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (133, 13)
X_test shape: (45, 13)


Standaryzacja

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)
X_train_std[:5]

array([[ 0.3342, -0.6358,  1.885 , -1.2365,  0.7672,  0.4867,  0.6193, -0.1488, -0.5162, -0.1736,  0.5559,  0.1695,  0.4312],
       [ 0.3816,  0.8185,  0.0717,  0.5915, -0.4822, -0.6441, -1.4178,  0.7699, -0.7097,  1.4536, -1.822 , -1.4818, -0.2815],
       [-0.7909, -1.2266, -1.6233, -1.4498,  2.5718, -0.6948, -0.2613, -0.0653,  1.9459, -0.6848,  0.4238, -0.4995,  0.0284],
       [-0.5778, -0.9993, -0.4407, -0.6272, -0.9681, -0.526 , -1.6087,  2.0228, -0.7097,  0.1885, -0.9413, -1.6384, -0.2815],
       [ 0.7961, -0.9902, -1.7416, -0.4749, -0.3434, -0.3572, -0.325 , -0.3159, -1.6242, -0.5144,  1.1724, -0.286 , -0.3434]])

PCA

In [7]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
X_train_pca = pca.fit_transform(X_train_std) #👈!!!!
X_test_pca = pca.transform(X_test_std)       #👈!!!!
X_train_pca.shape

(133, 3)

Wyjaśniona wariancja

In [8]:

results = pd.DataFrame(data={'explained_variance_ratio': pca.explained_variance_ratio_})
results['cumulative'] = results['explained_variance_ratio'].cumsum()
results['component'] = results.index + 1
results

,explained_variance_ratio,cumulative,component
0,0.356217,0.356217,1
1,0.196264,0.552481,2
2,0.111676,0.664157,3


In [9]:
fig = go.Figure(data=[go.Bar(x=results['component'], y=results['explained_variance_ratio'], name='explained variance ratio'),
                      go.Scatter(x=results['component'], y=results['cumulative'], name='cumulative explained variance')],
                layout=go.Layout(title=f'PCA - {pca.n_components_} components', width=950, template='plotly_dark'))
fig.show()

In [10]:
X_train_pca_df = pd.DataFrame(data=np.c_[X_train_pca, y_train], columns=['pca1', 'pca2', 'pca3', 'target'])
X_train_pca_df.head()

,pca1,pca2,pca3,target
0,-1.324483,-0.768185,0.756993,1.0
1,2.979035,-1.650972,-0.327724,3.0
2,-1.177306,0.425584,-1.263353,2.0
3,2.539467,0.173384,-1.041432,3.0
4,0.090910,0.997388,-1.893861,2.0


In [11]:
px.scatter_3d(X_train_pca_df, x='pca1', y='pca2', z='pca3', color='target', template='plotly_dark', width=950)

In [12]:
X_train_pca[:5]

array([[-1.3245, -0.7682,  0.757 ],
       [ 2.979 , -1.651 , -0.3277],
       [-1.1773,  0.4256, -1.2634],
       [ 2.5395,  0.1734, -1.0414],
       [ 0.0909,  0.9974, -1.8939]])

In [13]:
X_test_pca[:5]

array([[-2.3128, -1.2387, -0.9053],
       [-3.7372, -2.7523,  0.1145],
       [-2.4834, -1.8528, -0.2718],
       [-2.439 , -1.1071,  1.5526],
       [ 3.7613, -1.2835, -0.0479]])